### Práctica Final: Dashboard Tableau

#### Limpieza y preparado de datos en python

##### Fuente de datos 1: API idealista

*Como perdí el acceso a la API, algunas funciones no funcionan, por eso estan comentadas.

1. Carga librerias 

In [1]:
import pandas as pd
import json
import urllib
import requests as rq
import base64
import seaborn as sns
import matplotlib.pyplot as plt
import urllib3
from bs4 import BeautifulSoup
import re 

2. Funciones que obtienen la información de la API de idealista y la transforman en un data frame

In [ ]:
def get_oauth_token(): 
    
    # Esta función obtiene el token para acceder a la API de idealista usando unas credenciales,
    # que ellos te proporcionan por email.

    url = "https://api.idealista.com/oauth/token"    

    apikey= 'yxhs13m871hwge1svh83fnbu68zlwbh6' #proporcionado por idealista 
    secret= 'Ud4Le6J6xUzv'#proporcionado por idealista 
    apikey_secret = apikey + ':' + secret
    auth = str(base64.b64encode(bytes(apikey_secret, 'utf-8')))[2:][:-1]

    headers = {'Authorization' : 'Basic ' + auth,
               'Content-Type': 'application/x-www-form- urlencoded;charset=UTF-8'}
    params = urllib.parse.urlencode({'grant_type':'client_credentials'}) 
    content = rq.post(url,headers = headers, params=params)
    bearer_token = json.loads(content.text)['access_token']

    return bearer_token


def search_api(token, url):  
    
    # Esta función obtiene la conexión a la API pasandole como parámetros el token obtenido en 
    # get_oauth_token y la url en cuestión. 
    
    headers = {'Content-Type': 'Content-Type: multipart/form-data;', 'Authorization' : 'Bearer ' + token}
    content = rq.post(url, headers = headers)
    result = json.loads(content.text)

    return result

#parametros 
country = 'es' 
locale = 'es' 
language = 'es' 
max_items = '50'
#operation = 'sale' o 'rent'
property_type = 'homes'
order = 'priceDown' 
center = '40.4209832,-3.8196207' #MADRID 
distance = '60000'
sort = 'desc'
bankOffer = 'false'


def get_dataframe(operation):
    
    # Esta función transforma la información obtenida de la API en una dataframe para 
    # el análisis. (*Idealista solo permite un máximo de 
    # 50 items por petición y 100 peticiones al mes por el mismo usuario)
    
    df_tot = pd.DataFrame()
    limit = 10
    
    for i in range(1,limit):
        url = ('https://api.idealista.com/3.5/'+country+'/search?operation='+operation+"&locale="+locale+
               '&maxItems='+max_items+
               '&order='+order+
               '&center='+center+
               '&distance='+distance+
               '&propertyType='+property_type+
               '&sort='+sort+ 
               '&numPage=%s'+
               '&language='+language) % (i)  
        a = search_api(get_oauth_token(), url)
        df = pd.DataFrame.from_dict(a['elementList'])
        df_tot = pd.concat([df_tot,df])
    
    df_tot = df_tot.reset_index()
    df_tot = df_tot[['propertyCode','price', 'size', 'operation', 'rooms', 'bathrooms', 
                         'address', 'province', 'municipality', 'district', 'latitude', 
                         'longitude', 'priceByArea']]
    return df_tot


#token = get_oauth_token()
#url = "https://api.idealista.com/oauth/token"    
#search_api(token, url) ###### YA NO FUNCIONA PORQUE PERDI EL ACCESO A LA API DE IDEALISTA ######
#df_tot_ven = get_dataframe(operation = 'sale')
#df_tot_alq = get_dataframe(operation = 'rent')

3. Paso de los data frame a un output en formato excel

In [ ]:
#df_tot_ven = get_dataframe(operation = 'sale')
#df_tot_alq = get_dataframe(operation = 'rent')
#with pd.ExcelWriter('dat/output.xlsx') as writer:  
#    df_tot_ven.to_excel(writer, sheet_name='venta')
#    df_tot_alq.to_excel(writer, sheet_name='alquiler') 

* 4. Vuelvo a cargar los datos para introducir una nueva modificación y los exporto de nuevo a excel.

(como adjunto el docuemento **output.xlsx** creado cuando tenia acceso a la API, esta función si debería funcionar.

In [ ]:
def load_data():
    # Carga los datos, hace unas transformaciones y vuevle a exportar a excel
    
    # lectura de los datos 
    df_tot_ven = pd.read_excel('dat/output.xlsx', sheet_name = 'venta', index_col = 0)
    df_tot_alq = pd.read_excel('dat/output.xlsx', sheet_name = 'alquiler', index_col = 0)
    # calculo del precio medio por distritos
    precio_ven = df_tot_ven.groupby(by = ['District']).mean('Price')['Price'].reset_index()
    precio_alq = df_tot_alq.groupby(by = ['District']).mean('Price')['Price'].reset_index()
    # merge entre ambos dataframes para obtener uno con el precio medio 
    # de alquiler y venta por distritos
    precio_diff = precio_alq.merge(precio_ven, how = 'inner', 
                                   on = 'District', 
                                   suffixes=('_alq', '_ven'))
    # calculo de una nueva variable que nos de la rentabilidad anual de cada distrito
    precio_diff['rent'] = ((precio_diff['Price_alq'] * 12) / precio_diff['Price_ven'] ) * 100
    # exportacion a excel
    with pd.ExcelWriter('dat/output.xlsx') as writer:  
        df_tot_ven.to_excel(writer, sheet_name='venta')
        df_tot_alq.to_excel(writer, sheet_name='alquiler')
        precio_diff.to_excel(writer, sheet_name='rentabilidad')

load_data()

##### Fuente de datos 2: Serie temporal del INE.

In [3]:
def format_datetime(dataframe):
    # Carga datos, tranforma la fecha y exporta a excel
    # Cargo datos del csv
    trend = pd.read_csv('dat/trend.csv', sep = ';')
    
    # Transformo la variable trimestral para que pueda ser leida como formato fecha
    new = dataframe['Periodo'].str.split('T', n = 1, expand = True)
    dataframe['Anio'] = new[0]
    dataframe['Trimestre'] = new[1] 
    dataframe.drop(columns = ['Periodo'], inplace = True)
    dataframe['Trimestre'].replace({'4':'/10/01', 
                                    '3':'/07/01',
                                    '2':'/04/01',
                                    '1':'/01/01'}, inplace = True)
    dataframe['Fecha'] = dataframe['Anio'] + dataframe['Trimestre']
    dataframe['Fecha'] = pd.to_datetime(trend['Fecha'])
    dataframe.drop(columns = ['Anio', 'Trimestre'], inplace = True)
    
    # Los exporto a excel
    trend.to_excel('dat/trend.xlsx', index = False)
    
    
format_datetime(trend)